<a href="https://colab.research.google.com/github/ajna-del/ReStart/blob/master/DecisionsareTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#for timing
!pip install ipython-autotime
%load_ext autotime

#new labels 7000
# download = drive.CreateFile({'id': '1f1WDP7DiUMaDBoPjOmqj2Q-dS6JxCQhJ'})
# download.GetContentFile('NewLabels7000.7z') # Save content of this file as a local file - images
# os.mkdir('/content/RenamedImages')
# !7z e NewLabels7000.7z -o'/content/RenamedImages'

#new labels - undersampled 2308 images
download = drive.CreateFile({'id': '1wYWfu9enq8IBRtBa1iuRtZlOsXZSXURV'})
download.GetContentFile('CorrectLabelsUndersampled.7z') # Save content of this file as a local file - images
os.mkdir('/content/RenamedImages')
!7z e CorrectLabelsUndersampled.7z -o'/content/RenamedImages'

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=ef86004c28d315c5b7c1f26a64d48a6eda978a2d39f8915cdbfaeb83f72de0f0
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 548456742 bytes (524 MiB)

Extracting archive: Cor

In [0]:
os.mkdir('/content/RenamedImages/ClassN') 
os.mkdir('/content/RenamedImages/ClassO')
os.mkdir('/content/RenamedImages/ClassC') 
os.mkdir('/content/RenamedImages/ClassA') 
os.mkdir('/content/RenamedImages/ClassH') 
os.mkdir('/content/RenamedImages/ClassM')
os.mkdir('/content/RenamedImages/ClassD') 
os.mkdir('/content/RenamedImages/ClassG')
for filename in os.listdir('/content/RenamedImages'): # files extracted from zip file
  directory = '/content/RenamedImages'
  directoryN = '/content/RenamedImages/ClassN'
  directoryO = '/content/RenamedImages/ClassO'
  directoryC = '/content/RenamedImages/ClassC'
  directoryA = '/content/RenamedImages/ClassA'
  directoryH = '/content/RenamedImages/ClassH'
  directoryM = '/content/RenamedImages/ClassM'
  directoryD = '/content/RenamedImages/ClassD'
  directoryG = '/content/RenamedImages/ClassG'
  if filename.endswith('jpg'):
    if filename.startswith('N'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryN,filename))
    elif filename.startswith('O'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryO,filename))
    elif filename.startswith('C'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryC,filename))
    elif filename.startswith('A'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryA,filename))
    elif filename.startswith('H'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryH,filename))
    elif filename.startswith('M'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryM,filename))
    elif filename.startswith('D'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryD,filename))
    elif filename.startswith('G'):
      os.rename(os.path.join(directory,filename), os.path.join(directoryG,filename))

!pip install split-folders tqdm
import split_folders
split_folders.ratio('/content/RenamedImages', output="output", seed=333, ratio=(.8, .1, .1))

Copying files: 2308 files [00:02, 886.46 files/s]

time: 6.45 s


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([ transforms.Resize((500,500)),
                                transforms.ToTensor(), 
                                # transforms.Normalize(( 0.5,0.5,0.5), (0.5,0.5,0.5) )
                                ])

trainset = torchvision.datasets.ImageFolder(root='/content/output/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)#, collate_fn=my_collate,pin_memory=True))

testset = torchvision.datasets.ImageFolder(root='/content/output/test', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

valset = torchvision.datasets.ImageFolder(root='/content/output/val', target_transform=None)
valloader = torch.utils.data.DataLoader(valset, batch_size = 4,shuffle = True, num_workers = 2)
classes = ('Normal','Diseased','Glaucoma','Cataracts','AMD','Hypertension','Myopia','Other')


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(5, stride = 3)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 7)
        self.fc1 = nn.Linear(16* 52* 52, 996)
        self.fc2 = nn.Linear(996, 120)
        self.fc3 = nn.Linear(120, 8)
       

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.size())
        x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv3(x)))
        print(x.size())
        x = x.view(-1,  16* 52* 52)
        # print(x.size())
        x = F.relu(self.fc1(x))
        # print(x.size())
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [0]:
def normalize(batch):
  mean_red =0
  mean_green = 0
  mean_blue = 0
  std_red = 0
  std_green = 0
  std_blue = 0
  for i in range(batch.size(0)):
  #find rgb channel mean of image
    mean_red += batch[i, 0, :, :].mean().item()
    mean_green += batch[i, 1, :, :].mean().item()
    mean_blue += batch[i,2, :, :].mean().item()
    std_red += batch[i, 0, :, :].std()
    std_green += batch[i,  1, :, :].std()
    std_blue += batch[ i, 2, :, :].std()
  
  mean_red = mean_red /batch.size(0)
  mean_green = mean_green /batch.size(0)
  mean_blue = mean_blue /batch.size(0)
  std_red = std_red /batch.size(0)
  std_green = std_green /batch.size(0)
  std_blue = std_blue /batch.size(0)
  for i in range(batch.size(0)):
    batch[i, 0, :, :] = (batch[i, 0, :, :] -mean_red)/std_red
    batch[i, 1, :, :] = (batch[i, 1, :, :] -mean_green)/std_green
    batch[i, 2, :, :] = (batch[i, 2, :, :] -mean_blue)/std_blue
  return batch

for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = normalize(inputs)
        print('labels:   ',labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        print('predicted:',predicted)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    print('Accuracy of the network on the 100(0) training images: %d %%' % (
        100 * correct / total))
print('Finished Training')

labels:    tensor([1, 0, 1, 3])
torch.Size([4, 16, 52, 52])
predicted: tensor([2, 2, 2, 2])
labels:    tensor([7, 4, 3, 6])
torch.Size([4, 16, 52, 52])
predicted: tensor([1, 1, 1, 1])
labels:    tensor([6, 7, 1, 7])
torch.Size([4, 16, 52, 52])
predicted: tensor([0, 0, 3, 4])
labels:    tensor([6, 2, 4, 0])
torch.Size([4, 16, 52, 52])
predicted: tensor([6, 7, 7, 7])
labels:    tensor([3, 0, 0, 7])
torch.Size([4, 16, 52, 52])
predicted: tensor([6, 7, 2, 2])
labels:    tensor([6, 3, 4, 3])
torch.Size([4, 16, 52, 52])
predicted: tensor([0, 2, 0, 7])
labels:    tensor([1, 5, 3, 5])
torch.Size([4, 16, 52, 52])
predicted: tensor([0, 0, 0, 0])
labels:    tensor([3, 7, 4, 4])
torch.Size([4, 16, 52, 52])
predicted: tensor([6, 6, 6, 0])
labels:    tensor([1, 3, 5, 2])
torch.Size([4, 16, 52, 52])
predicted: tensor([4, 6, 6, 6])
labels:    tensor([3, 5, 1, 3])
torch.Size([4, 16, 52, 52])
predicted: tensor([6, 3, 4, 6])
labels:    tensor([2, 6, 1, 6])
torch.Size([4, 16, 52, 52])
predicted: tensor([6

KeyboardInterrupt: ignored

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torch.Size([4, 6, 664, 331])
torch.Size([4, 16, 219, 108])
torch.Size([4, 378432])
torc